In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils import data
import time
from DNN import Dataset, train_epoch, eval_epoch, eval_epoch_true, FC, Seq2Seq, Dataset_res 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Data for time

In [ ]:
training_data = torch.load("data_time/training_time.pt").float()
test_data = torch.load("data_time/test_time.pt").float()

In [ ]:
training_data.shape, test_data.shape 

### Normalization 

In [ ]:
mean = torch.tensor([training_data[:, 0, :, :].mean(), training_data[:, 1, :, :].mean(), training_data[:, 2, :, :].mean()])
std = torch.tensor([training_data[:, 0, :, :].std(), training_data[:, 1, :, :].std(), training_data[:, 2, :, :].std()])
training_norm = torch.zeros(training_data.shape, dtype = torch.float) 
training_norm[:, 0, :, :] = (training_data[:, 0, :, :] - mean[0]) / std[0]
training_norm[:, 1, :, :] = (training_data[:, 1, :, :] - mean[1]) / std[1]
training_norm[:, 2, :, :] = (training_data[:, 2, :, :] - mean[2]) / std[2] 

test_norm = torch.zeros(test_data.shape, dtype = torch.float) 
test_norm[:, 0, :, :] = (test_data[:, 0, :, :] - mean[0]) / std[0]
test_norm[:, 1, :, :] = (test_data[:, 1, :, :] - mean[1]) / std[1]
test_norm[:, 2, :, :] = (test_data[:, 2, :, :] - mean[2]) / std[2]

In [ ]:
training_set = Dataset(training_norm)
test_set = Dataset(test_norm) 
training_set, val_set = data.random_split(training_set, [int(len(training_set) * 0.875), int(len(training_set) - int(len(training_set) * 0.875))])

In [ ]:
# data loader 
train_loader = data.DataLoader(training_set, batch_size = 512, shuffle = True)
val_loader = data.DataLoader(val_set, batch_size = 512, shuffle = False) 
test_loader = data.DataLoader(test_set, batch_size = 512, shuffle = False) 

In [ ]:
print(len(training_set))
print(len(val_set))
print(len(test_set))

### Training 

In [ ]:
# train model 

trial_num = 3  
preds_total = [] 
test_losses = [] 
num_epoch = 100 

for i in range(trial_num): 
    # build model 
    model = Seq2Seq(input_dim = 234, hidden_dim = 512, output_dim = 234, num_layers = 1).to(device) 
#     model = FC(input_dim = 60, input_len = 12, hidden_dim = 256, output_dim = 60).to(device) 
#     model = Latent_ODE(latent_dim = 256, obs_dim = 240, nhidden = 512, rhidden = 512, aug = False).to(device)
    name = "Seq2Seq"
    learning_rate = 0.01
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 1, gamma=0.95)
    criterion = nn.MSELoss()
    print(sum(p.numel() for p in model.parameters() if p.requires_grad))
    print("------Trial", i + 1)
    best_loss = 100   
    train_losses = []
    val_losses = [] 
    
    for epoch in range(1, num_epoch + 1): 
        start = time.time()
        train_loss = train_epoch(model, train_loader, optimizer, criterion)[-1]
        train_losses.append(train_loss)
        _, _, val_loss = eval_epoch(model, val_loader, criterion) 
        val_losses.append(val_loss)
        if val_loss <= best_loss: 
            best_loss = val_loss 
            best_model = model 
            torch.save({"preds": preds, "trues": trues, "model": best_model}, "best_space_" + name + str(i+1) + ".pt") 
        end = time.time()
        print("Epoch:", epoch, "completed in:", (end - start), "s. Training loss:", train_loss, ". Val loss:", val_loss) 
        if (len(train_losses) > 50 and np.mean(val_losses[-5:]) >= np.mean(val_losses[-10:-5])):
            break
        scheduler.step() 
        if epoch % 5 == 0: print(optimizer.param_groups[0]['lr']) 
    
    # save the best model, prediction and ground truth 
    preds, trues, test_loss = eval_epoch_true(best_model, test_loader, criterion, std, mean) 
    preds_total.append(preds)
    test_losses.append(test_loss)
    print(test_loss)  

### Evaluation 

In [ ]:
rmse_losses = []
for preds_i in preds_total: 
#     print(preds_i.shape)
    rmse_losses.append(torch.sqrt(criterion(torch.tensor(preds_i), torch.tensor(trues))))

In [ ]:
rmse_losses

In [ ]:
np.array(rmse_losses).mean()

In [ ]:
# sensor location 

xi = [0,  45,  56,  75,  81,  86,  89,  95, 100, 105, 109, 112, 117,
       124, 128, 133, 137, 141, 146, 149, 152, 158, 163, 167, 171, 174,
       180, 186, 192, 197, 200, 205, 207, 210, 211, 213, 214, 228, 231,
       237, 240, 242, 251, 254, 258, 262, 266, 270, 277, 279, 282, 283,
       286, 288, 291, 294, 296, 298, 300, 303, 308, 310, 315, 317, 320,
       322, 327, 338, 342, 345, 352, 356, 359, 362, 366, 368, 374, 379] 

In [ ]:
## Plot the last time step from the first test sample 

preds_t = np.stack(preds_total)
preds_mean = preds_t.mean(axis = 0)
gt = trues[0]
loss = np.array(test_losses).mean()

In [ ]:
std_1 = preds_t.std(axis = 0)[-1, 0, -1]
std_2 = preds_t.std(axis = 0)[-1, 1, -1]
std_3 = preds_t.std(axis = 0)[-1, 2, -1]

In [ ]:
result = preds_mean[-1, :, -1, :]
std = [std_1, std_2, std_3]
true = trues[-1][:, -1, :]

In [ ]:
import matplotlib.pyplot as plt

tp=24
plt.style.use('default')
label={0: "Density $k$ (veh/m)",
       1: "Flow $q$ (veh/s)",
       2: "Speed $u$ (m/s)"} 

from matplotlib import gridspec

fwyp=[xi[n]* 300 / 1.e3 for n in range(len(xi))]

#gs = gridspec.GridSpec(3, 1, height_ratios=[2, 1, 1]) 
gs_kw={"height_ratios": [3, 3, 3]}

fig4,ax4=plt.subplots(figsize=(5, 15), nrows=3, gridspec_kw=gs_kw, sharex=True)
fig4.subplots_adjust(left=0.08, bottom=0.08, right=0.98, top=0.95)
ymin=[-0.01, -0.1, 20]
ymax=[0.06, 1.5, 40]


for n in range(3): 
    ax4[n].plot(fwyp, (true[n, :]), label='observed', marker='o', markersize=5)
    ax4[n].plot(fwyp, (result[n, :]), label='predicted', marker='o', markersize=5)
    ax4[n].fill_between(fwyp, (result[n, :] - std[n]), (result[n, :] + std[n]),  color='red', alpha=.3)    
    ax4[n].set_ylim(ymin[n], ymax[n])
    #ax4[n].set_xlabel("Distance (grid points)")
    ax4[n].set_ylabel(label[n])

    #ax4[n].set_xlim(fwyp[0]-1, fwyp[-1]+1)

#     lastpoint=-100
#     if n==0:
#         for p in range(len(xi)):
#             if (fwyp[p] - lastpoint) > 1.:
#                 ax4[n].text(fwyp[p], (y_pred[n, :, tp-68])[p] + 0.02, "none", rotation=90, fontsize=8)
#                 lastpoint=fwyp[p]
ax4[0].set_title("FC: last timestep from the last test sample")    
ax4[2].set_xlabel("Relative distance from North to South (km)")
#ax4[n].set_ylabel("Traffic density (Veh/m)")
fig4.savefig('FC.jpg', dpi=300)